In [1]:
import time

import torch
import transformers
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
)


e:\dev\auray\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Declare model

In [2]:
model_name = "Breeze-7B-32k-Instruct-v1_0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    low_cpu_mem_usage=True,  # try to limit RAM
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),  # load model in low precision to save memory
    # attn_implementation="flash_attention_2",
)
# Building a LLM QNA chain
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    streamer=streamer,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.42s/it]


In [3]:
llama_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Read Markdown File

In [4]:
start_time = time.time()

# Loading and splitting the document
loader = UnstructuredMarkdownLoader("CNS16190-zh_TW.md")
data = loader.load()
print(data)

print("\ntime: %.2fs\n" % (time.time() - start_time))


[Document(page_content='中華民國國家標準 CNS\n\n消費者物聯網之網宇安全：基準要求事項 Cybersecurity for consumer internet of things: Baseline requirements\n\nCNS16190:2023\n\nX6147\n\n中華民國112年1月13日制定公布\n\nDate of Promulgation:2023-01-13\n\n中華民國 年 月 日修訂公布\n\nDate of Amendment: - -\n\n目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3\n\n1.適用範圍 4\n\n2.引用標準 4\n\n3.用語及定義、符號及縮寫 4\n\n4.報告實作 9\n\n5.消費者IoT裝置之網宇安全控制措施 10\n\n5.1無通用之預設通行碼 10\n\n5.2實作管理脆弱性報告之方式 11\n\n5.3保持軟體為最新 12\n\n5.4安全儲存敏感性安全參數 15\n\n5.5安全通訊 16\n\n5.6最小化暴露之攻擊面 17\n\n5.7確保軟體完整性 19\n\n5.8確保個人資料安全 19\n\n5.9使系統對中斷具韌性 20\n\n5.10檢查系統遙測資料 20\n\n5.11使用者容易刪除使用者資料 21\n\n5.12使裝置容易安裝及維護 21\n\n5.13驗核輸入資料 22\n\n6.消費者IoT裝置之資料保護控制措施 22\n\n附錄A(參考)基本概念及模型 23\n\n附錄B(參考)實作符合性聲明一覽表 28\n\n名詞對照表 36\n\n參考資料 36\n\n前言\n\n本標準係依標準法之規定，經國家標準審查委員會審定，由主管機關公布之中華民國國家標準。\n\n依標準法第四條之規定，國家標準採自願性方式實施。但經各該目的事業主管機關引用全部或部分內容為法規者，從其規定。\n\n本標準並未建議所有安全事項，使用本標準前應適當建立相關維護安全與健康作業，並且遵守相關法規之規定。\n\n本標準之部分內容，可能涉及專利權、商標權與著作權，主管機關及標準專責機關不負責任何或所有此類專利權、商標權與著作權之鑑別。\n\n簡介\n\n當家庭中更多裝置連接至網際網路時，物聯網(internet of things,IoT)之網宇安全(cybers

### CharacterTextSplitter

In [5]:
start_time = time.time()

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(data)
for chunk in chunked_documents:
    print(chunk)

print("\ntime: %.2fs\n" % (time.time() - start_time))


Created a chunk of size 1129, which is longer than the specified 500


page_content='中華民國國家標準 CNS\n\n消費者物聯網之網宇安全：基準要求事項 Cybersecurity for consumer internet of things: Baseline requirements\n\nCNS16190:2023\n\nX6147\n\n中華民國112年1月13日制定公布\n\nDate of Promulgation:2023-01-13\n\n中華民國 年 月 日修訂公布\n\nDate of Amendment: - -\n\n目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3\n\n1.適用範圍 4\n\n2.引用標準 4\n\n3.用語及定義、符號及縮寫 4\n\n4.報告實作 9\n\n5.消費者IoT裝置之網宇安全控制措施 10\n\n5.1無通用之預設通行碼 10\n\n5.2實作管理脆弱性報告之方式 11\n\n5.3保持軟體為最新 12\n\n5.4安全儲存敏感性安全參數 15\n\n5.5安全通訊 16\n\n5.6最小化暴露之攻擊面 17\n\n5.7確保軟體完整性 19\n\n5.8確保個人資料安全 19\n\n5.9使系統對中斷具韌性 20\n\n5.10檢查系統遙測資料 20\n\n5.11使用者容易刪除使用者資料 21' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='5.7確保軟體完整性 19\n\n5.8確保個人資料安全 19\n\n5.9使系統對中斷具韌性 20\n\n5.10檢查系統遙測資料 20\n\n5.11使用者容易刪除使用者資料 21\n\n5.12使裝置容易安裝及維護 21\n\n5.13驗核輸入資料 22\n\n6.消費者IoT裝置之資料保護控制措施 22\n\n附錄A(參考)基本概念及模型 23\n\n附錄B(參考)實作符合性聲明一覽表 28\n\n名詞對照表 36\n\n參考資料 36\n\n前言\n\n本標準係依標準法之規定，經國家標準審查委員會審定，由主管機關公布之中華民國國家標準。\n\n依標準法第四條之規定，國家標準採自願性方式實施。但經各該目的事業主管機關引用全部或部分內容為法規者，從其規定。\n\n本標準並未建議所有安全事項，使用本標準前應適當建立相關維護安全與健康作業，並且遵守

### RecursiveCharacterTextSplitter

In [6]:
start_time = time.time()

# Chunk text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=0,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "",
        "。",
        "，",
        "、",
        "；",

    ],
)
chunked_documents = text_splitter.split_documents(data)
for chunk in chunked_documents:
    print(chunk)

print("\ntime: %.2fs\n" % (time.time() - start_time))


page_content='中華民國國家標準 CNS' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='消費者物聯網之網宇安全：基準要求事項 Cybersecurity for' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='consumer internet of things: Baseline' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='requirements' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='CNS16190:2023\n\nX6147' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='中華民國112年1月13日制定公布' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='Date of Promulgation:2023-01-13' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='中華民國 年 月 日修訂公布\n\nDate of Amendment: - -' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3\n\n1.適用範圍 4' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='2.引用標準 4\n\n3.用語及定義、符號及縮寫 4\n\n4.報告實作 9' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='5.消費者IoT裝置之網宇安全控制措施 10' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='5.1無通用之預設通行碼 10\n\n5.2實作管理脆弱性報告之方式 11' metadata={'so

### MarkdownTextSplitter

In [7]:
start_time = time.time()

# Chunk text
text_splitter = MarkdownTextSplitter(chunk_size=40, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(data)
for chunk in chunked_documents:
    print(chunk)

print("\ntime: %.2fs\n" % (time.time() - start_time))

page_content='中華民國國家標準 CNS' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='消費者物聯網之網宇安全：基準要求事項 Cybersecurity for' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='consumer internet of things: Baseline' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='requirements' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='CNS16190:2023\n\nX6147' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='中華民國112年1月13日制定公布' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='Date of Promulgation:2023-01-13' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='中華民國 年 月 日修訂公布\n\nDate of Amendment: - -' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3\n\n1.適用範圍 4' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='2.引用標準 4\n\n3.用語及定義、符號及縮寫 4\n\n4.報告實作 9' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='5.消費者IoT裝置之網宇安全控制措施 10' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='5.1無通用之預設通行碼 10\n\n5.2實作管理脆弱性報告之方式 11' metadata={'so

### MarkdownTextSplitter & RecursiveCharacterTextSplitter

In [13]:
start_time = time.time()

# Chunk text
md_splits = MarkdownTextSplitter().split_documents(data)
print(f'md_splits len: {len(md_splits)}')
# print(md_splits[0])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=30)
chunked_documents = text_splitter.split_documents(md_splits)
print(f'chunked_documents len: {len(chunked_documents)}')
for chunk in chunked_documents:
    print(chunk)

print("\ntime: %.2fs\n" % (time.time() - start_time))


md_splits len: 9
chunked_documents len: 1195
page_content='中華民國國家標準 CNS' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='消費者物聯網之網宇安全：基準要求事項 Cybersecurity for consumer' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='Cybersecurity for consumer internet of things:' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='consumer internet of things: Baseline' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='internet of things: Baseline requirements' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='CNS16190:2023\n\nX6147\n\n中華民國112年1月13日制定公布' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='Date of Promulgation:2023-01-13\n\n中華民國 年 月 日修訂公布' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='中華民國 年 月 日修訂公布\n\nDate of Amendment: - -\n\n目錄' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='Date of Amendment: - -\n\n目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3' metadata={'source': 'CNS16190-zh_TW.md'}
page_content='目錄\n\n節次 頁次\n\n前言 2\n\n簡介 3\n\n1.適用範圍 4\n\n2.引用標準 4' met

### Build Retriever

In [19]:
start_time = time.time()

# Load chunked documents into the Qdrant index
db = Qdrant.from_documents(
    chunked_documents,
    HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1"),
    location=":memory:", # Local mode with in-memory storage only
)

retriever = db.as_retriever(search_type="mmr")
retrieval_chain = RetrievalQA.from_llm(llm=llama_llm, retriever=retriever)

print("\ntime: %.2fs\n" % (time.time() - start_time))



time: 102.75s



In [20]:
query = "用中文摘要cns16190"

start_time = time.time()
response = retrieval_chain.invoke(query)

# Extract only the necessary part from the response
# start_idx = response.find("Helpful Answer: ")
# if start_idx != -1:
#     response = response[start_idx + len("Helpful Answer: ") :]

print(f'response: {response}')
print("\nInference time: %.2fs\n" % (time.time() - start_time))

「CNS16190」是指中華民國（臺灣）在2023年1月13日制定公布的一项标准。这项标准涉及到使用遵循相关ETSI（European Telecommunications Standards Institute）标准的UICC（Universal Integrated Circuit Card），主要集中在如何使系统对中斷具韌</s>
response: {'query': '用中文摘要cns16190', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\nCNS16190:2023\n\nX6147\n\n中華民國112年1月13日制定公布\n\nContext:\n5.9 使系統對中斷具韌性\n\nContext:\n於遵循相關ETSI標準的UICC中(例：參照ETSITS102221[25])。\n\nContext:\nebofThings[18]中。此等裝置無需鑑別即可存取，從而對所有人開放存取。\n\nQuestion: 用中文摘要cns16190\nHelpful Answer: 「CNS16190」是指中華民國（臺灣）在2023年1月13日制定公布的一项标准。这项标准涉及到使用遵循相关ETSI（European Telecommunications Standards Institute）标准的UICC（Universal Integrated Circuit Card），主要集中在如何使系统对中斷具韌"}

Inference time: 32.56s

